In [1]:
import numpy as np
import open3d as o3d
import pandas as pd
from sklearn.cluster import KMeans
import random
import os
import glob
from sklearn.naive_bayes import BernoulliNB
from sklearn.cluster import KMeans

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def preprocess_point_cloud(pcd):
    pcd = pcd.voxel_down_sample(voxel_size=0.02)

    radius_normal = 0.2
    print("estimating normals")
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = 0.5
    print("computing fpfh")
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    fpfh = np.asarray(pcd_fpfh.data).T
    return pcd, fpfh

In [3]:
def cov_metrics(pcd):
    print("estimating covariances")
    covs = pcd.estimate_covariances()
    covs = np.asarray(pcd.covariances)

    print("computing eigen-based metrics")
    metrics = []
    for pt in covs:
        eigenvalues = np.linalg.eigvals(pt)
        e1, e2, e3 = eigenvalues
        
        linearity = (e1 - e2) / e1
        planarity = (e2 - e3) / e1
        scattering = e3 / e1
        omnivariance = (e1 * e2 * e3) ** (1 / 3)
        anisotropy = (e1 - e3) / e1
        eigentropy = -(e1 * np.log(e1) + e2 * np.log(e2) + e3 * np.log(e3))
        curvature = e3 / (e1 + e2 + e3)

        metrics.append((linearity, planarity, scattering, omnivariance, anisotropy, eigentropy, curvature))

    dtype = [('linearity', 'f8'), ('planarity', 'f8'), ('scattering', 'f8'), 
            ('omnivariance', 'f8'), ('anisotropy', 'f8'), ('eigentropy', 'f8'), 
            ('curvature', 'f8')]
    
    metrics_array = np.array(metrics, dtype=dtype)
  
    return np.array([tuple(row) for row in metrics_array])

In [4]:
def get_clusters(df):
    df['cluster'] = KMeans(n_clusters=4,
                           random_state=0,
                           init=initial_centroids).fit(df).labels_
    return df

In [5]:
print("reading point cloud")
pcd = o3d.io.read_point_cloud("C:/Users/ellie/OneDrive/Desktop/lidar_local/ccb-3_preprocessed.pcd")
print("preprocessing point cloud")
pcd, fpfh = preprocess_point_cloud(pcd)
print("getting metrics")
cov = cov_metrics(pcd)

cov_headers = ['linearity', 'planarity', 'scattering', 'omnivariance', 'anisotropy', 'eigentropy', 'curvature']
header = ['x', 'y', 'z'] + [f'feature{i}' for i in range(fpfh.shape[1])] + cov_headers
all_metrics = np.hstack([np.asarray(pcd.points), fpfh, cov])

df = pd.DataFrame(all_metrics, columns=header)

reading point cloud
preprocessing point cloud
estimating normals
computing fpfh
getting metrics
estimating covariances
computing eigen-based metrics


In [6]:
cov_headers = ['linearity', 'planarity', 'scattering', 'omnivariance', 'anisotropy', 'eigentropy', 'curvature']
header = ['x', 'y', 'z'] + [f'feature{i}' for i in range(fpfh.shape[1])] + cov_headers
all_metrics = np.hstack([np.asarray(pcd.points), fpfh, cov])
df = pd.DataFrame(all_metrics, columns=header)

In [7]:
df = df[df['z'] <= 0.25]
df['cluster'] = KMeans(n_clusters=3, random_state=0).fit(df).labels_
print(df)

                x        y         z   feature0   feature1   feature2  \
2        4.905200 -0.43500  0.046400  13.666005  24.332968   7.588833   
3        1.139750 -5.37725 -0.177000  10.355552   0.064889   0.000000   
4       -8.092500 -4.88075 -0.781250   0.635878   0.000000   0.000000   
5       -2.220333 -0.27700 -0.178667  19.358223  20.594219  10.085666   
7        0.255000  0.73600 -0.153250  26.001741   0.865008   0.326415   
...           ...      ...       ...        ...        ...        ...   
3235342 -6.218000  3.98500 -0.619000   3.158911   0.505319   0.200013   
3235343 -6.274000  4.01100 -0.616000   8.294866   1.142367   0.378043   
3235344 -6.294000  4.01700 -0.605000  12.175871   4.792762   0.843039   
3235345 -8.119000  4.90000 -0.677000   7.205642   0.061322   0.065501   
3235346 -7.673000  4.60600 -0.386000   2.675847   8.224408   9.266978   

          feature3   feature4    feature5   feature6  ...  feature31  \
2         7.179208  26.230699   63.518802  23.39720

In [8]:
from open3d.web_visualizer import draw

just_clusters = df[["x", "y", "z", "cluster"]]
cluster_0 = just_clusters[just_clusters['cluster'] == 0]
cluster_1 = just_clusters[just_clusters['cluster'] == 1]
cluster_2 = just_clusters[just_clusters['cluster'] == 2]

zero = cluster_0[['x', 'y', 'z']].values
one = cluster_1[['x', 'y', 'z']].values
two = cluster_2[['x', 'y', 'z']].values

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(two)
draw(pcd2)

[Open3D INFO] Resetting default logger to print to terminal.


WebVisualizer(window_uid='window_0')